In [1]:
import json,re,random
import pandas as pd
import spacy
from spacy import displacy
import nltk
import dataframe_image as dfi
nltk.download('averaged_perceptron_tagger')
nlp = spacy.load("en_core_web_sm")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\zzm\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [27]:
#download if haven't install the package
# !pip install -U pip setuptools wheel
# !pip install -U spacy
# !python -m spacy download zh_core_web_sm
!python -m spacy download en_core_web_trf

[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')


## 3.2 Most frequent ⟨ Noun - Adjective ⟩ pairs for each rating.
Each review has a “star” rating in the range of
1 to 5. Randomly select 50 reviews (one from each business) of rating 1, extract the top-10 most frequent
noun-adjective pairs from the sentences in these selected reviews. Example noun-adjective pairs are servicegreat,
food-delicious, that appear in the same sentence. Do the same for 20 reviews of ratings 2, 3, 4, and 5,
respectively. Discuss your results and limitations of your method.

In [2]:
#list to string method
def list_to_string(list_):
    str1 = "" 
    
    # traverse in the string  
    for ele in list_: 
        str1 += ele + " "  
    
    # return string  
    return str1 

### load reviews and preprocess functions

In [3]:
data_set = []
#load yelp reviews.json
for line in open('yelp_academic_dataset_review.json', 'rb'):
    data_set.append(json.loads(line))

In [4]:
#function that select reviews based on stars(from different business)
def stars_reviews(dataset,num_of_reviews,stars):
    business_list = []
    reviews_ = []
    while len(reviews_) < num_of_reviews:
        ran = random.randint(0, len(dataset)-1)
        if data_set[ran]['business_id'] not in business_list:
            business_list.append(data_set[ran]['business_id'])
            
            if data_set[ran]['stars'] == stars:
                reviews_.append(data_set[ran]['text'])
    print(f"randomly selected {len(reviews_)} reviews of star {stars} in different {num_of_reviews} business")
    return reviews_

In [5]:
print(len(data_set))

8635403


### functions of calculating pair frequency

In [6]:
def find_conj_noun(noun, noun_list):
    noun_list.append(noun.lemma_)
    for t in noun.children:
        if t.pos_ == 'NOUN'and t.dep_ == 'conj' and t.n_rights == 0:
            noun_list.append(t.lemma_)
            return noun_list 
        elif t.pos_ == 'NOUN'and t.dep_ == 'conj' and t.n_rights != 0:
            return find_conj_noun(t,noun_list)   
    return noun_list

def find_conj_adj(adj,adj_list):
    adj_list.append(adj.text)
    for t in adj.children:
        if t.pos_ == 'ADJ'and t.dep_ == 'conj' and t.n_rights == 0:
            adj_list.append(t.text)
            return adj_list 
        elif t.pos_ == 'ADJ'and t.dep_ == 'conj' and t.n_rights != 0:
            return find_conj_adj(t,adj_list)
    return adj_list

def dict_insertion(s,dic):
    if s in dic:
        dic[s] += 1
    else:
        dic[s] = 1

#main function of finding noun-adj pair frequency dictionary
def noun_adj_freqdict(s):
    noun_adj_dict = {} 
    new_s = ""
    noun_list = []
    adj_list = []
    doc = nlp(s)
    for token in doc:
        if token.pos_ == 'ADJ':
            
            if token.dep_== 'amod':
                if token.head.pos_ =='NOUN':
                    noun_list = find_conj_noun(token.head, noun_list)
                adj_list = find_conj_adj(token,adj_list)
                if noun_list != None:
                    for i in noun_list:
                        for j in adj_list:
                            new_s = (i + "-" + j).lower()                               
                            dict_insertion(new_s,noun_adj_dict)
                noun_list=[]
                adj_list =[]
                
            if token.dep_ == 'acomp':
                adj_list = find_conj_adj(token,adj_list)
                for t in token.head.lefts:
                    if t.pos_ == 'NOUN':
                        noun_list = find_conj_noun(t, noun_list)
                        if noun_list != None:
                            for i in noun_list:
                                for j in adj_list:
                                    new_s = (i + "-" + j).lower()                               
                                    dict_insertion(new_s,noun_adj_dict)
                noun_list=[]
                adj_list =[]           
    sorted_dict = {}
    sorted_keys = sorted(noun_adj_dict, key=noun_adj_dict.get, reverse=True)

    for w in sorted_keys:
        sorted_dict[w] = noun_adj_dict[w]           
    return sorted_dict

### result: 50 reviews (one from each business) of rating 1

In [24]:
r50_1_raw = stars_reviews(data_set,50,1.0)

randomly selected 50 reviews of star 1.0 in different 50 business


In [25]:
r50_1 = list_to_string(r50_1_raw).replace("\n","")

In [26]:
r50_1_result = noun_adj_freqdict(r50_1)
r50_1_df = pd.DataFrame.from_dict(r50_1_result,orient='index',columns=['frequency'])

dfi.export(r50_1_df.head(10), 'r50_1_df.png')
r50_1_df.head(10)

,frequency
food-good,5
experience-worst,4
day-next,3
desk-front,3
time-first,3
food-great,2
service-worst,2
wear-normal,2
order-online,2
table-other,2


### result: 20 reviews (one from each business) of rating 2

In [27]:
r20_2_raw = stars_reviews(data_set,20,2.0)

randomly selected 20 reviews of star 2.0 in different 20 business


In [28]:
r20_2 = list_to_string(r20_2_raw).replace("\n","")

In [29]:
r20_2_result = noun_adj_freqdict(r20_2)
r20_2_df = pd.DataFrame.from_dict(r20_2_result,orient='index',columns=['frequency'])

dfi.export(r20_2_df.head(10), 'r20_2_df.png')
r20_2_df.head(10)

,frequency
time-last,2
place-many,2
food-fast,2
haircut-great,2
language-different,1
stand-hostess,1
greasy-little,1
staff-friendly,1
chair-giant,1
chair-comfy,1


### result: 20 reviews (one from each business) of rating 3

In [30]:
r20_3_raw = stars_reviews(data_set,20,3.0)

randomly selected 20 reviews of star 3.0 in different 20 business


In [31]:
r20_3 = list_to_string(r20_3_raw).replace("\n","")

In [32]:
r20_3_result = noun_adj_freqdict(r20_3)
r20_3_df = pd.DataFrame.from_dict(r20_3_result,orient='index',columns=['frequency'])
dfi.export(r20_3_df.head(10), 'r20_3_df.png')
r20_3_df.head(10)

,frequency
hour-happy,4
music-more,3
music-local,3
food-good,2
place-good,2
music-new,2
music-different,2
spot-good,2
experience-perfect,1
food-bad,1


### result: 20 reviews (one from each business) of rating 4

In [33]:
r20_4_raw = stars_reviews(data_set,20,4.0)

randomly selected 20 reviews of star 4.0 in different 20 business


In [34]:
r20_4 = list_to_string(r20_4_raw).replace("\n","")

In [35]:
r20_4_result = noun_adj_freqdict(r20_4)
r20_4_df = pd.DataFrame.from_dict(r20_4_result,orient='index',columns=['frequency'])
dfi.export(r20_4_df.head(10), 'r20_4_df.png')
r20_4_df.head(10)

,frequency
breakfast-good,2
place-full,2
place-great,2
spot-great,2
food-good,2
line-long,2
cappuccino-best,1
day-other,1
birthday-happy,1
job-horrible,1


### result: 20 reviews (one from each business) of rating 5

In [36]:
r20_5_raw = stars_reviews(data_set,20,5.0)

randomly selected 20 reviews of star 5.0 in different 20 business


In [37]:
r20_5 = list_to_string(r20_5_raw).replace("\n","")

In [38]:
r20_5_result = noun_adj_freqdict(r20_5)
r20_5_df = pd.DataFrame.from_dict(r20_5_result,orient='index',columns=['frequency'])
dfi.export(r20_5_df.head(10), 'r20_5_df.png')
r20_5_df.head(10)

,frequency
service-friendly,2
time-first,2
place-favorite,2
food-whole,1
experience-great,1
lesson-interesting,1
lesson-mini,1
people-outstanding,1
food-good,1
side-fresh,1
